In [19]:
import numpy as np
import cv2
import glob
import csv
import matplotlib.pyplot as plt
import tensorflow as tf

ModuleNotFoundError: No module named 'cv2'

In [2]:
images = {} #image_file:RGB
for file in glob.glob("./image/*"):
    im = cv2.imread(file) # BGR
    images[file] = im

NameError: name 'glob' is not defined

In [3]:
print(len(images.values()))
print(list(images.values())[0].shape)
plt.imshow(list(images.values())[0][:,:,[2,1,0]]) # conver to RGB

0


IndexError: list index out of range

In [4]:
keys = list(images.keys())
for k in keys:
    square_length = min(len(images[k]), len(images[k][0]))
    images[k] = images[k][:square_length,:square_length,:]
    compress = int(square_length / 100)
    images[k] = images[k][::compress,::compress,[2,1,0]]
    if images[k].shape != (100, 100, 3):
        print(images[k].shape)
        images[k] = images[k][:100,:100,:]

In [5]:
import sys
csv.field_size_limit(sys.maxsize)


X_data = [] #image
Y_data = [] #subjectivity

i = 0
with open('./imagemodel_dataset.csv', 'r') as csvreader:
    reader = csv.DictReader(csvreader)
    for row in reader:
        file_name = "./image/" + row['id'] + ".jpg"
        target = row['subjectivity']
        try:
            X_data.append(images[file_name])
            Y_data.append(float(target))
            i += 1
        except:
            print(file_name)
        if i == 150:
            break

NameError: name 'csv' is not defined

In [6]:
print(len(X_data) == len(Y_data))
print(len(X_data))

NameError: name 'X_data' is not defined

In [7]:
# INPUT SHAPE
print(X_data[0].shape)
# OUTPUT SHAPE
print(Y_data[0])

NameError: name 'X_data' is not defined

In [8]:
plt.imshow(X_data[75])

NameError: name 'plt' is not defined

In [9]:
Y_data_norm = [(a-min(Y_data))/(max(Y_data) - min(Y_data)) for a in Y_data]
plt.hist(Y_data_norm, bins = 20)

NameError: name 'Y_data' is not defined

In [10]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(128,(3,3), activation="relu", input_shape=X_data[0].shape))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(.1))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(100,activation="sigmoid"))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(1))

NameError: name 'tf' is not defined

In [11]:
model.compile(loss=tf.keras.losses.mean_squared_error,
             optimizer=tf.keras.optimizers.Adam(),
             metrics=['mse'])

NameError: name 'model' is not defined

In [12]:
X_data_norm = tf.cast(np.asarray(X_data), tf.float64)/255
X_train = X_data_norm[5:]
Y_train = Y_data_norm[5:]
X_test = X_data_norm[:5]
Y_test = Y_data_norm[:5]

NameError: name 'tf' is not defined

In [13]:
plt.imshow(X_test[1])
print(Y_test[1])

NameError: name 'plt' is not defined

In [14]:
model.fit(X_train, np.asarray(Y_train),
        validation_data=(np.asarray(X_test), np.asarray(Y_test)),
        batch_size = 32,
        epochs=4,
        verbose=2)

NameError: name 'model' is not defined

In [16]:
plt.imshow(X_test[1])
print(Y_test[1])

NameError: name 'plt' is not defined

In [ ]:
Y_predict = model.predict(tf.cast(np.asarray(X_test), tf.float64))
print(Y_predict)